# Reducir dimensionalidad a conjuntos de datos con formato libsvm

In [1]:
# Import SparkSession
import findspark
findspark.init()

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.sql import SparkSession
import random
import sys
#path = str(sys.argv[1])
#atr = int(sys.argv[2])

In [2]:
# Build the SparkSession
spark = SparkSession.builder \
    .master("local[6]") \
    .appName("Data exploration URL") \
    .config("spark.executor.memory", "4gb") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
sc._conf.getAll()

[('spark.app.id', 'local-1615387882159'),
 ('spark.executor.memory', '4gb'),
 ('spark.driver.port', '36269'),
 ('spark.driver.memory', '4g'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'Data exploration URL'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.master', 'local[6]'),
 ('spark.driver.host', 'fedora')]

In [30]:
data = sc.textFile("../data/url_svmlight/Dimension_General_x_500.svm")

In [31]:
data.take(1)

['1 2:1 4:0.0746888 5:0.124138 6:0.0588235 10:1 11:0.142857 17:0.922497 18:0.859345 19:0.229479 21:0.285713 22:0.0238095 23:0.0238095 24:1 25:1 33:0.0555556 41:0.1 44:1 45:1 47:1 62:1 64:1 66:1 68:1 70:1 72:1 74:1 76:1 82:1 84:1 86:1 88:1 90:1 92:1 94:1 96:1 102:1 104:1 106:1 108:1 110:1 112:1 139:1 141:1 143:1 145:1 149:1']

In [14]:
# Metodo que guarda cada renglon en el archivo .svm
def save_file(data, limit):
    file = open('../data/url_svmlight/Dimension_General_x_' +str(limit) + '.svm', 'a')
    file.write('\n' + data)
    file.close()

In [15]:
# Metodo para reducir la dimension, a traves de un ciclo va recorriendo cada posicion de la instancia hasta
# llegar al limite establecido, en casos particulares donde la ultima posicion este lejos del limite, se 
# almacena llamando al metodo save_file() despues de haberse cumplido la condicion del 3er if() anidado.50000
def reduce_dimension(row, limit):
    instance = row.split(' ')
    data = instance[0]

    for feature in range(len(instance)):
        if (feature > 0): 
            pos_aux = instance[feature].split(':')
            if (int(pos_aux[0]) <= limit):  
                data += ' ' + str(instance[feature])
                if (len(instance) - 1 == feature):
                    save_file(data, limit)
            else:
                save_file(data, limit)
                data = ''
                # print('Se rompe el ciclo')
                break

In [32]:
new_data = data.map(lambda row: reduce_dimension(row, 100)) # El 2o. argumento es el limite de caracteristicas.

In [33]:
new_data.count()

2396130

In [13]:
pwd

'/home/jsarabia/Documents/IA/Data-exploration-url_svmlight/code'

In [10]:
# Celda de prueba:
# instance = data.take(1)
# limit_dimension = 80
# reduce_dimension(instance, limit_dimension)